### Euler Problem 129

A number consisting entirely of ones is called a repunit. We shall define R(k) to be a repunit of length k; for example, R(6) = 111111.

Given that d is a positive integer and GCD(d, 10) = 1, it can be shown that there always exists a value, k, for which R(k) is divisible by d, and let A(d) be the least such value of k; for example, A(7) = 6 and A(41) = 5.

The least value of d for which A(d) first exceeds ten is 17.

Find the least value of d for which A(d)first exceeds one-million.

In [550]:
using Primes, Lazy, Combinatorics

`Printing Utilities`

In [3]:
macro print_seq(aseq) 
    :(for p in $aseq println(p) end) 
end

@print_seq (macro with 1 method)

In [4]:
macro print_seq(aseq, lines)
    :(for p in take($aseq,$lines) println(p) end)
end

@print_seq (macro with 2 methods)

`stats(dict) returns attributes of dict`

In [5]:
function stats(dict)
    e = last(sort(collect(keys(dict))))
    #m = last(sort(collect(values(dict))))
    return("Dict[$e] = $(dict[e]), Size = $(length(dict))")
end

stats (generic function with 1 method)

`sorted_dict(dict) returns a sorted array of dictionary pairs`

In [6]:
sorted_dict(dict) = [(k,dict[k]) for k in sort(collect(keys(dict)))]

sorted_dict (generic function with 1 method)

`ds(n) = GCD(d, 10) = 1 | d<-1:inf`

In [7]:
ds = @>> Lazy.range(2) filter(x->gcd(x,10)==1);

In [8]:
take(10,ds)

(3 7 9 11 13 17 19 21 23 27)

`order(R) returns the size of R(k), i.e., k`

In [9]:
order(R) = floor(Integer, log10(R)) + 1

order (generic function with 1 method)

`R(k) returns Repunit of size k`

In [10]:
R(k) = div(BigInt(10)^k - 1 ,9)

R (generic function with 1 method)

In [11]:
R(12)

111111111111

In [12]:
function make_R1s(n)
    n<1 && return(0)
    Rs = Dict()
    k,r1,r2 = 1,BigInt(1),BigInt(0)
    Rs[k] = r1
    while(k<n)
        k,r1,r2 = (k+1),(11*r1 - 10*r2), r1
        Rs[k] = r1
        #println("$k, $r1, $r2")        
    end
    return(Rs)
end    

make_R1s (generic function with 1 method)

In [13]:
@time R1s = make_R1s(10^5);
order(R1s[10^5])

  2.484166 seconds (1.14 M allocations: 5.825 GiB, 22.03% gc time)


100000

Though R1(n) is significantly slower than R(n) it is able to generate R1[k] from R1[k-1] and R1[k-2]. This is extremely important. For example R(10^5) runs in 0.002 seconds vs R1(10^5) which requires 1.2 seconds for R1(10^5), roughly a 500X difference. For Rs(10^5) and R1s(10^5), the run times were 17.5 seconds and the same 1.2 seconds. This is a 15X performance difference in favor of the slower R1(k).

The recursion equation is: `R[k] = 11*R[k-1] - 10*R[k-2]`

`Rs_lazy returns a lazy sequence of R(n) | n <- 1:inf`

In [14]:
Rs_lazy = @>> Lazy.range(1) map(R);

`make_A_goal(Rs, goal) returns A[d], where goal is the maximum divisor and Rs is sequence of R(k), including a lazy sequence`

In [15]:
#Original ds over Rs
function make_A_goal(Rs,goal)
    A = Dict()
    for d in ds
        for (k,R) in enumerate(Rs)
            if mod(R,d) == 0
                A[d]= k
                if A[d] > goal
                    println(stats(A))
                    return(A)
                end
                break
            end
        end
    end
    return("Failed")
end

make_A_goal (generic function with 1 method)

In [194]:
@time A = make_A_goal(Rs_lazy,2*10^4);

Dict[20047] = 20046, Size = 8018
 48.718932 seconds (171.51 M allocations: 3.104 GiB, 12.72% gc time)


`first_k(Rs, d) returns the k of the first (smallest) d that divides, where Rs is a sequence of R(k)| k<-1..n`

In [17]:
function first_k(Rs,d) 
    for (k,R) in enumerate(Rs)
        if R%d == 0
            return(k)
        end
    end
    return("nothing")
end

first_k (generic function with 1 method)

In [18]:
println(first_k(Rs_lazy,11))

2


`make_A_size(n) returns A[d] of size n`

In [19]:
make_A_size(n) = Dict(@>> ds map(d->(d => first_k(Rs_lazy,d))) take(n))

make_A_size (generic function with 1 method)

In [20]:
@time A = make_A_size(4002);
println(stats(A))

  8.527138 seconds (52.22 M allocations: 953.386 MiB, 9.51% gc time)
Dict[10007] = 10006, Size = 4002


In [21]:
A_d = [(d,k) for (d,k) in sort(collect(pairs(A)))]
println(" d, k")
@print_seq A_d

 d, k
(3, 3)
(7, 6)
(9, 9)
(11, 2)
(13, 6)
(17, 16)
(19, 18)
(21, 6)
(23, 22)
(27, 27)
(29, 28)
(31, 15)
(33, 6)
(37, 3)
(39, 6)
(41, 5)
(43, 21)
(47, 46)
(49, 42)
(51, 48)
(53, 13)
(57, 18)
(59, 58)
(61, 60)
(63, 18)
(67, 33)
(69, 66)
(71, 35)
(73, 8)
(77, 6)
(79, 13)
(81, 81)
(83, 41)
(87, 84)
(89, 44)
(91, 6)
(93, 15)
(97, 96)
(99, 18)
(101, 4)
(103, 34)
(107, 53)
(109, 108)
(111, 3)
(113, 112)
(117, 18)
(119, 48)
(121, 22)
(123, 15)
(127, 42)
(129, 21)
(131, 130)
(133, 18)
(137, 8)
(139, 46)
(141, 138)
(143, 6)
(147, 42)
(149, 148)
(151, 75)
(153, 144)
(157, 78)
(159, 39)
(161, 66)
(163, 81)
(167, 166)
(169, 78)
(171, 18)
(173, 43)
(177, 174)
(179, 178)
(181, 180)
(183, 60)
(187, 16)
(189, 54)
(191, 95)
(193, 192)
(197, 98)
(199, 99)
(201, 33)
(203, 84)
(207, 198)
(209, 18)
(211, 30)
(213, 105)
(217, 30)
(219, 24)
(221, 48)
(223, 222)
(227, 113)
(229, 228)
(231, 6)
(233, 232)
(237, 39)
(239, 7)
(241, 30)
(243, 243)
(247, 18)
(249, 123)
(251, 50)
(253, 22)
(257, 256)
(259, 6)
(261, 

(2073, 690)
(2077, 165)
(2079, 54)
(2081, 1040)
(2083, 1041)
(2087, 298)
(2089, 1044)
(2091, 240)
(2093, 66)
(2097, 2088)
(2099, 2098)
(2101, 190)
(2103, 2100)
(2107, 42)
(2109, 18)
(2111, 1055)
(2113, 2112)
(2117, 56)
(2119, 162)
(2121, 12)
(2123, 192)
(2127, 708)
(2129, 532)
(2131, 710)
(2133, 351)
(2137, 2136)
(2139, 330)
(2141, 2140)
(2143, 2142)
(2147, 1008)
(2149, 306)
(2151, 63)
(2153, 2152)
(2157, 1077)
(2159, 336)
(2161, 30)
(2163, 102)
(2167, 98)
(2169, 90)
(2171, 498)
(2173, 65)
(2177, 930)
(2179, 2178)
(2181, 726)
(2183, 174)
(2187, 2187)
(2189, 198)
(2191, 312)
(2193, 336)
(2197, 1014)
(2199, 183)
(2201, 105)
(2203, 1101)
(2207, 2206)
(2209, 2162)
(2211, 66)
(2213, 553)
(2217, 246)
(2219, 474)
(2221, 2220)
(2223, 18)
(2227, 1040)
(2229, 2226)
(2231, 1056)
(2233, 84)
(2237, 1118)
(2239, 1119)
(2241, 1107)
(2243, 1121)
(2247, 318)
(2249, 258)
(2251, 2250)
(2253, 375)
(2257, 60)
(2259, 450)
(2261, 144)
(2263, 120)
(2267, 1133)
(2269, 2268)
(2271, 27)
(2273, 2272)
(2277, 198)


(4159, 693)
(4161, 72)
(4163, 1980)
(4167, 1386)
(4169, 378)
(4171, 672)
(4173, 318)
(4177, 4176)
(4179, 198)
(4181, 336)
(4183, 1012)
(4187, 13)
(4189, 2030)
(4191, 42)
(4193, 1794)
(4197, 699)
(4199, 144)
(4201, 75)
(4203, 2097)
(4207, 300)
(4209, 660)
(4211, 4210)
(4213, 382)
(4217, 4216)
(4219, 4218)
(4221, 198)
(4223, 170)
(4227, 96)
(4229, 4228)
(4231, 2115)
(4233, 1968)
(4237, 666)
(4239, 702)
(4241, 1060)
(4243, 2121)
(4247, 120)
(4249, 606)
(4251, 108)
(4253, 1063)
(4257, 126)
(4259, 4258)
(4261, 4260)
(4263, 84)
(4267, 400)
(4269, 474)
(4271, 2135)
(4273, 1424)
(4277, 138)
(4279, 388)
(4281, 2139)
(4283, 2141)
(4287, 1428)
(4289, 2144)
(4291, 102)
(4293, 1053)
(4297, 1432)
(4299, 4296)
(4301, 176)
(4303, 330)
(4307, 232)
(4309, 690)
(4311, 2151)
(4313, 2034)
(4317, 2157)
(4319, 264)
(4321, 1036)
(4323, 390)
(4327, 4326)
(4329, 18)
(4331, 420)
(4333, 618)
(4337, 4336)
(4339, 4338)
(4341, 1446)
(4343, 84)
(4347, 594)
(4349, 4348)
(4351, 684)
(4353, 870)
(4357, 242)
(4359, 726)


(6411, 2136)
(6413, 286)
(6417, 990)
(6419, 2730)
(6421, 2140)
(6423, 6420)
(6427, 1071)
(6429, 2142)
(6431, 3132)
(6433, 918)
(6437, 390)
(6439, 184)
(6441, 1008)
(6443, 3024)
(6447, 306)
(6449, 1612)
(6451, 2150)
(6453, 189)
(6457, 586)
(6459, 6456)
(6461, 210)
(6463, 308)
(6467, 3108)
(6469, 924)
(6471, 3231)
(6473, 6472)
(6477, 336)
(6479, 90)
(6481, 270)
(6483, 30)
(6487, 498)
(6489, 306)
(6491, 1298)
(6493, 525)
(6497, 88)
(6499, 1056)
(6501, 294)
(6503, 1392)
(6507, 270)
(6509, 3102)
(6511, 3056)
(6513, 498)
(6517, 882)
(6519, 195)
(6521, 815)
(6523, 592)
(6527, 3180)
(6529, 1088)
(6531, 930)
(6533, 46)
(6537, 2178)
(6539, 1506)
(6541, 30)
(6543, 2178)
(6547, 1091)
(6549, 174)
(6551, 3275)
(6553, 6552)
(6557, 533)
(6559, 936)
(6561, 6561)
(6563, 3281)
(6567, 198)
(6569, 1642)
(6571, 6570)
(6573, 312)
(6577, 2192)
(6579, 1008)
(6581, 1316)
(6583, 3164)
(6587, 2820)
(6589, 598)
(6591, 1014)
(6593, 3114)
(6597, 549)
(6599, 3299)
(6601, 330)
(6603, 105)
(6607, 2202)
(6609, 1101)
(66

(8687, 48)
(8689, 2172)
(8691, 8688)
(8693, 4346)
(8697, 222)
(8699, 8698)
(8701, 336)
(8703, 2898)
(8707, 4353)
(8709, 8706)
(8711, 420)
(8713, 8712)
(8717, 4158)
(8719, 4359)
(8721, 432)
(8723, 60)
(8727, 8724)
(8729, 84)
(8731, 8730)
(8733, 105)
(8737, 2912)
(8739, 8730)
(8741, 8740)
(8743, 624)
(8747, 4373)
(8749, 672)
(8751, 1458)
(8753, 8752)
(8757, 414)
(8759, 4140)
(8761, 876)
(8763, 462)
(8767, 398)
(8769, 39)
(8771, 3738)
(8773, 705)
(8777, 4290)
(8779, 22)
(8781, 8778)
(8783, 8782)
(8787, 84)
(8789, 368)
(8791, 4292)
(8793, 8784)
(8797, 1386)
(8799, 1254)
(8801, 1014)
(8803, 1467)
(8807, 8806)
(8809, 4202)
(8811, 396)
(8813, 3774)
(8817, 8814)
(8819, 8818)
(8821, 8820)
(8823, 2064)
(8827, 96)
(8829, 324)
(8831, 4415)
(8833, 88)
(8837, 4418)
(8839, 4419)
(8841, 420)
(8843, 21)
(8847, 8838)
(8849, 553)
(8851, 2158)
(8853, 678)
(8857, 208)
(8859, 984)
(8861, 8860)
(8863, 8862)
(8867, 4433)
(8869, 1260)
(8871, 4434)
(8873, 4194)
(8877, 804)
(8879, 2046)
(8881, 2173)
(8883, 1242)

`R_over_ds(R,ds) returns the divisors of R`

In [22]:
R_over_ds(R,ds) = filter(d->R%d==0,ds)

R_over_ds (generic function with 1 method)

`B(n) returns the ds that divide R(k)`

In [24]:
#original
make_B(n) = Dict(
    @>> Rs_lazy take(n) map(R->(order(R),R)) map(p->(p[1] =>R_over_ds(p[2],take(Integer(ceil(.42*n)),ds)))) filter(p-> !isempty(p[2])))

make_B (generic function with 1 method)

In [25]:
@time B = make_B(43);
println(length(B))

  0.498350 seconds (1.11 M allocations: 55.114 MiB)
31


In [26]:
@print_seq [(k,B[k]) for k in sort(collect(keys(B)))]

(2, (11))
(3, (3 37))
(4, (11))
(5, (41))
(6, (3 7 11 13 21 33 37 39))
(8, (11))
(9, (3 9 37))
(10, (11 41))
(12, (3 7 11 13 21 33 37 39))
(14, (11))
(15, (3 31 37 41))
(16, (11 17))
(18, (3 7 9 11 13 19 21 33 37 39))
(20, (11 41))
(21, (3 37 43))
(22, (11 23))
(24, (3 7 11 13 21 33 37 39))
(25, (41))
(26, (11))
(27, (3 9 27 37))
(28, (11 29))
(30, (3 7 11 13 21 31 33 37 39 41))
(32, (11 17))
(33, (3 37))
(34, (11))
(35, (41))
(36, (3 7 9 11 13 19 21 33 37 39))
(38, (11))
(39, (3 37))
(40, (11 41))
(42, (3 7 11 13 21 33 37 39 43 49))


In [27]:
make_B(n) = Dict(
    @>> Rs_lazy take(n) map(R->(order(R),R)) map(p->(p[1] =>R_over_ds(p[2],take(Integer(ceil(.42*n)),ds)))) filter(p-> !isempty(p[2])))

make_B (generic function with 1 method)

In [28]:
#Original
function make_B_lazy(n)
    n<1 && return(0)
    B = Dict()
    ds_set = take(Integer(ceil(.42*n)),ds)
    #println(">>$ds_set")
    k,R,R2 = 1,BigInt(1),BigInt(0)
    # B[k] = divisors_of_R[k]
    while(k<n)
        k,R,R2 = (k+1),(11*R - 10*R2), R
        rds = R_over_ds(R,ds_set)
        if !isempty(rds)
            B[k] = rds
            #println("$k =>$rds")
        end
    end
    println(stats(B))
    return(B)
end    

make_B_lazy (generic function with 1 method)

In [29]:
@print_seq sort_dict(make_B_lazy(100))

Dict[100] = (11 41 101), Size = 78


UndefVarError: UndefVarError: sort_dict not defined

In [30]:
function make_B2A(n)
    n<1 && return(0)
    B = Dict()
    ds_set = take(Integer(ceil(.42*n)),ds)
    #println(">>$ds_set")
    k,R,R2 = 1,BigInt(1),BigInt(0)
    # B[k] = divisors_of_R[k]
    while(k<n)
        k,R,R2 = (k+1),(11*R - 10*R2), R
        rds = R_over_ds(R,ds_set)
        if !isempty(rds)
            #println("$k =>$rds")
            for d in rds
                if !haskey(B,d)
                    B[d] = k
                    #println("*$(d)")
                end
            end
        end
    end
    #println(stats(B))
    return(B)
end    

make_B2A (generic function with 1 method)

In [31]:
@time B2A = make_B2A(2*10^3);

  1.739430 seconds (16.99 M allocations: 340.427 MiB)


In [32]:
@print_seq filter(p-> mod(p[1],p[2]) == 1,[(k,d) for (k,d) in sort(collect(pairs(B2A)))])

(7, 6)
(11, 2)
(13, 6)
(17, 16)
(19, 18)
(23, 22)
(29, 28)
(31, 15)
(37, 3)
(41, 5)
(43, 21)
(47, 46)
(53, 13)
(59, 58)
(61, 60)
(67, 33)
(71, 35)
(73, 8)
(79, 13)
(83, 41)
(89, 44)
(91, 6)
(97, 96)
(101, 4)
(103, 34)
(107, 53)
(109, 108)
(113, 112)
(127, 42)
(131, 130)
(137, 8)
(139, 46)
(149, 148)
(151, 75)
(157, 78)
(163, 81)
(167, 166)
(173, 43)
(179, 178)
(181, 180)
(191, 95)
(193, 192)
(197, 98)
(199, 99)
(211, 30)
(223, 222)
(227, 113)
(229, 228)
(233, 232)
(239, 7)
(241, 30)
(251, 50)
(257, 256)
(259, 6)
(263, 262)
(269, 268)
(271, 5)
(277, 69)
(281, 28)
(283, 141)
(293, 146)
(307, 153)
(311, 155)
(313, 312)
(317, 79)
(331, 110)
(337, 336)
(347, 173)
(349, 116)
(353, 32)
(359, 179)
(367, 366)
(373, 186)
(379, 378)
(383, 382)
(389, 388)
(397, 99)
(401, 200)
(409, 204)
(419, 418)
(421, 140)
(431, 215)
(433, 432)
(439, 219)
(443, 221)
(449, 32)
(451, 10)
(457, 152)
(461, 460)
(463, 154)
(467, 233)
(479, 239)
(481, 6)
(487, 486)
(491, 490)
(499, 498)
(503, 502)
(509, 508)
(521, 52)

In [33]:
function B2A(B)
    A = Dict()
    for k in sort(collect(keys(B)))
        ds = B[k]
        for d in ds
            if !haskey(A,d)
                A[d] = k
            end
        end
    end
    println(stats(A))
    return(A)
end
#sort(collect(pairs(AA)))

ErrorException: cannot define function B2A; it already has a value

In [34]:
@time B_lazy = make_B_lazy(10^3);

Dict[1000] = (11 41 73 101 137 251 271 401 451 751 803), Size = 823
  1.658702 seconds (883.04 k allocations: 18.204 MiB, 59.42% gc time)


In [35]:
A_kds = Dict([(k, sort(map(first,ds))) for (k,ds) in groupby(last,A)])
A_kds_list = [(k,ds) for (k,ds) in sort(pairs(A_kds))]
@print_seq A_kds_list
length(A_kds_list)

(2, [11])
(3, [3, 37, 111])
(4, [101, 1111])
(5, [41, 271])
(6, [7, 13, 21, 33, 39, 77, 91, 143, 231, 259, 273, 407, 429, 481, 777, 1001, 1221, 1443, 2849, 3003, 3367, 5291, 8547])
(7, [239, 4649])
(8, [73, 137, 803, 1507, 7373, 10001])
(9, [9, 333])
(10, [451, 2981, 9091])
(12, [303, 707, 1313, 2121, 3333, 3737, 3939, 7777, 9191, 9901])
(13, [53, 79, 4187])
(14, [2629])
(15, [31, 93, 123, 813, 1147, 1271, 1517, 3441, 3813, 4551, 8401])
(16, [17, 187, 1241, 1717, 2329])
(18, [19, 57, 63, 99, 117, 133, 171, 209, 247, 399, 627, 693, 703, 741, 819, 1197, 1287, 1463, 1729, 1881, 2109, 2223, 2331, 2717, 3663, 4329, 4389, 4921, 5187, 6327, 7733, 8151, 9009, 9139])
(20, [3541, 4141])
(21, [43, 129, 717, 1591, 1933, 4773, 5799, 8843])
(22, [23, 121, 253, 2783, 4093, 8779])
(24, [219, 411, 511, 949, 959, 1533, 1781, 2409, 2701, 2847, 2877, 4521, 5069, 5343, 5621, 6643, 8103])
(26, [583, 859, 869, 9449])
(27, [27, 757, 999, 2271, 6813])
(28, [29, 281, 319, 2929, 3091, 6931, 8149])
(29, [3191])
(

(390, [393, 917, 1703, 2751, 4061, 4323, 4847, 4867, 5109, 5239, 6437, 6929])
(391, [7039])
(393, [787, 2361])
(396, [801, 1691, 5073, 5607, 8811])
(398, [8767])
(399, [4791])
(400, [4267, 6817])
(402, [7237])
(403, [1613])
(405, [2511, 3321, 5053, 6683])
(408, [2857, 8571])
(409, [1637])
(411, [7401])
(413, [827, 4957])
(414, [423, 893, 1251, 2641, 2679, 2961, 4653, 5263, 5499, 6251, 7923, 8037, 8757, 9823])
(418, [419, 4609, 9637])
(419, [839])
(420, [899, 1263, 1769, 2623, 2697, 2947, 2989, 3567, 4331, 5307, 5473, 6119, 6177, 6293, 6989, 7747, 7869, 8323, 8711, 8841, 8967, 9889])
(423, [1693, 2547, 5079])
(424, [7811])
(425, [2551])
(426, [5971, 9383])
(429, [3551, 5293])
(430, [1721, 4741])
(431, [3449])
(432, [433, 459, 1299, 1853, 3031, 3213, 3897, 4763, 5049, 5559, 5629, 5967, 7361, 8227, 8721, 9093])
(433, [5197])
(438, [877, 879, 2051, 2631, 3073, 3809, 4829, 5707, 6139, 6153, 9219, 9647, 9669])
(440, [881, 9691])
(441, [883, 2649, 7947])
(442, [4873, 5459, 8137])
(443, [6203]

(1365, [2733, 8191])
(1366, [1367])
(1368, [4113, 8683])
(1369, [5477])
(1380, [1381, 1383, 2867, 3227, 4143, 5993, 8479, 8601, 9661, 9667, 9681])
(1386, [4167, 8797, 8901, 9751])
(1392, [2787, 3009, 5569, 6503, 7021])
(1394, [8549])
(1395, [2799, 8649])
(1398, [3269, 6071, 9793, 9807])
(1400, [2801])
(1401, [2803, 8409])
(1404, [5777, 8611])
(1419, [2841])
(1421, [2843])
(1422, [6023])
(1423, [5693])
(1424, [3043, 4273])
(1428, [1429, 4287, 7143, 8961, 10003])
(1429, [5717])
(1430, [3013])
(1431, [2889])
(1432, [1433, 4297])
(1433, [8599])
(1434, [3353, 6227])
(1435, [5893])
(1439, [2879])
(1446, [1447, 4341])
(1450, [5801])
(1456, [5989, 8927, 9299])
(1458, [2917, 5103, 8019, 8751, 9477])
(1462, [5849])
(1466, [7331])
(1467, [8803])
(1470, [1473, 3437, 6107, 6383])
(1478, [2957])
(1481, [2963])
(1484, [3103])
(1486, [1487])
(1487, [8923])
(1488, [6341])
(1494, [1503, 3173, 4491, 8973, 9481, 9519])
(1499, [2999])
(1500, [3001, 9003])
(1504, [4513])
(1506, [1509, 3521, 6539])
(1516, [4

(3996, [4023])
(4002, [8319])
(4006, [4007])
(4016, [8551])
(4018, [4019])
(4019, [8039])
(4020, [8339])
(4026, [8053, 8441])
(4038, [8079])
(4046, [8093])
(4048, [8993])
(4050, [4051])
(4056, [4057])
(4061, [8123])
(4065, [8133])
(4072, [4073])
(4073, [8147])
(4080, [8959])
(4090, [4091])
(4094, [8413])
(4095, [8199])
(4098, [4099, 4101, 9569])
(4104, [8209])
(4115, [8231])
(4118, [8237])
(4121, [8243])
(4126, [4127])
(4138, [4139])
(4140, [4149, 8507, 8759])
(4152, [4153])
(4155, [8311])
(4158, [8717])
(4176, [4177, 8361, 8891, 9027])
(4181, [8363])
(4184, [8369])
(4185, [8397])
(4194, [8873])
(4200, [8403])
(4202, [8809])
(4210, [4211])
(4215, [8431])
(4216, [4217])
(4218, [4219])
(4221, [8443])
(4228, [4229])
(4233, [8467])
(4257, [8523])
(4258, [4259])
(4260, [4261])
(4263, [8529])
(4268, [8947])
(4272, [9129])
(4278, [9989])
(4281, [8563])
(4284, [8903])
(4286, [8573])
(4290, [8777, 9039])
(4292, [8791])
(4293, [8667])
(4296, [4299])
(4302, [9101])
(4313, [8627])
(4317, [8637])
(

1851

`k_over(A,goal) returns the lowest (k > goal) in A`

In [36]:
function k_over(A,goal) 
    for d in sort(collect(keys(A)))
        if A[d] > goal
            return((goal, d, A[d]))
            break
        end
    end
    println("nothing")
end

k_over (generic function with 1 method)

In [238]:
for g in 10:10:100
    println("$(k_over(A,g))")
end

(10, 17, 16)
(20, 23, 22)
(30, 47, 46)
(40, 47, 46)
(50, 59, 58)
(60, 69, 66)
(70, 81, 81)
(80, 81, 81)
(90, 97, 96)
(100, 109, 108)


`factors(n) printable version of factor(n)`

In [37]:
factors(n) = join([v==1 ? "$k" : "$k^$v" for (k,v) in pairs(factor(n))], " ⋅ ")

factors (generic function with 1 method)

In [38]:
for k in 2:37
    println("$k=> $(factors(R(k)))")
end

2=> 11
3=> 3 ⋅ 37
4=> 11 ⋅ 101
5=> 41 ⋅ 271
6=> 3 ⋅ 7 ⋅ 11 ⋅ 13 ⋅ 37
7=> 239 ⋅ 4649
8=> 11 ⋅ 73 ⋅ 101 ⋅ 137
9=> 3^2 ⋅ 37 ⋅ 333667
10=> 11 ⋅ 41 ⋅ 271 ⋅ 9091
11=> 21649 ⋅ 513239
12=> 3 ⋅ 7 ⋅ 11 ⋅ 13 ⋅ 37 ⋅ 101 ⋅ 9901
13=> 53 ⋅ 79 ⋅ 265371653
14=> 11 ⋅ 239 ⋅ 4649 ⋅ 909091
15=> 3 ⋅ 31 ⋅ 37 ⋅ 41 ⋅ 271 ⋅ 2906161
16=> 11 ⋅ 17 ⋅ 73 ⋅ 101 ⋅ 137 ⋅ 5882353
17=> 2071723 ⋅ 5363222357
18=> 3^2 ⋅ 7 ⋅ 11 ⋅ 13 ⋅ 19 ⋅ 37 ⋅ 52579 ⋅ 333667
19=> 1111111111111111111
20=> 11 ⋅ 41 ⋅ 101 ⋅ 271 ⋅ 3541 ⋅ 9091 ⋅ 27961
21=> 3 ⋅ 37 ⋅ 43 ⋅ 239 ⋅ 1933 ⋅ 4649 ⋅ 10838689
22=> 11^2 ⋅ 23 ⋅ 4093 ⋅ 8779 ⋅ 21649 ⋅ 513239
23=> 11111111111111111111111
24=> 3 ⋅ 7 ⋅ 11 ⋅ 13 ⋅ 37 ⋅ 73 ⋅ 101 ⋅ 137 ⋅ 9901 ⋅ 99990001
25=> 41 ⋅ 271 ⋅ 21401 ⋅ 25601 ⋅ 182521213001
26=> 11 ⋅ 53 ⋅ 79 ⋅ 859 ⋅ 265371653 ⋅ 1058313049
27=> 3^3 ⋅ 37 ⋅ 757 ⋅ 333667 ⋅ 440334654777631
28=> 11 ⋅ 29 ⋅ 101 ⋅ 239 ⋅ 281 ⋅ 4649 ⋅ 909091 ⋅ 121499449
29=> 3191 ⋅ 16763 ⋅ 43037 ⋅ 62003 ⋅ 77843839397
30=> 3 ⋅ 7 ⋅ 11 ⋅ 13 ⋅ 31 ⋅ 37 ⋅ 41 ⋅ 211 ⋅ 241 ⋅ 271 ⋅ 2161 ⋅ 9091 ⋅ 29

In [195]:
A_d = [(d,k) for (d,k) in sort(collect(pairs(A)))]
println(" d, k")
@print_seq A_d

 d, k
(3, 3)
(7, 6)
(9, 9)
(11, 2)
(13, 6)
(17, 16)
(19, 18)
(21, 6)
(23, 22)
(27, 27)
(29, 28)
(31, 15)
(33, 6)
(37, 3)
(39, 6)
(41, 5)
(43, 21)
(47, 46)
(49, 42)
(51, 48)
(53, 13)
(57, 18)
(59, 58)
(61, 60)
(63, 18)
(67, 33)
(69, 66)
(71, 35)
(73, 8)
(77, 6)
(79, 13)
(81, 81)
(83, 41)
(87, 84)
(89, 44)
(91, 6)
(93, 15)
(97, 96)
(99, 18)
(101, 4)
(103, 34)
(107, 53)
(109, 108)
(111, 3)
(113, 112)
(117, 18)
(119, 48)
(121, 22)
(123, 15)
(127, 42)
(129, 21)
(131, 130)
(133, 18)
(137, 8)
(139, 46)
(141, 138)
(143, 6)
(147, 42)
(149, 148)
(151, 75)
(153, 144)
(157, 78)
(159, 39)
(161, 66)
(163, 81)
(167, 166)
(169, 78)
(171, 18)
(173, 43)
(177, 174)
(179, 178)
(181, 180)
(183, 60)
(187, 16)
(189, 54)
(191, 95)
(193, 192)
(197, 98)
(199, 99)
(201, 33)
(203, 84)
(207, 198)
(209, 18)
(211, 30)
(213, 105)
(217, 30)
(219, 24)
(221, 48)
(223, 222)
(227, 113)
(229, 228)
(231, 6)
(233, 232)
(237, 39)
(239, 7)
(241, 30)
(243, 243)
(247, 18)
(249, 123)
(251, 50)
(253, 22)
(257, 256)
(259, 6)
(261, 

(2141, 2140)
(2143, 2142)
(2147, 1008)
(2149, 306)
(2151, 63)
(2153, 2152)
(2157, 1077)
(2159, 336)
(2161, 30)
(2163, 102)
(2167, 98)
(2169, 90)
(2171, 498)
(2173, 65)
(2177, 930)
(2179, 2178)
(2181, 726)
(2183, 174)
(2187, 2187)
(2189, 198)
(2191, 312)
(2193, 336)
(2197, 1014)
(2199, 183)
(2201, 105)
(2203, 1101)
(2207, 2206)
(2209, 2162)
(2211, 66)
(2213, 553)
(2217, 246)
(2219, 474)
(2221, 2220)
(2223, 18)
(2227, 1040)
(2229, 2226)
(2231, 1056)
(2233, 84)
(2237, 1118)
(2239, 1119)
(2241, 1107)
(2243, 1121)
(2247, 318)
(2249, 258)
(2251, 2250)
(2253, 375)
(2257, 60)
(2259, 450)
(2261, 144)
(2263, 120)
(2267, 1133)
(2269, 2268)
(2271, 27)
(2273, 2272)
(2277, 198)
(2279, 273)
(2281, 228)
(2283, 1140)
(2287, 762)
(2289, 108)
(2291, 364)
(2293, 1146)
(2297, 2296)
(2299, 198)
(2301, 174)
(2303, 966)
(2307, 192)
(2309, 2308)
(2311, 231)
(2313, 2304)
(2317, 330)
(2319, 579)
(2321, 30)
(2323, 44)
(2327, 534)
(2329, 16)
(2331, 18)
(2333, 583)
(2337, 90)
(2339, 2338)
(2341, 2340)
(2343, 210)
(

(4227, 96)
(4229, 4228)
(4231, 2115)
(4233, 1968)
(4237, 666)
(4239, 702)
(4241, 1060)
(4243, 2121)
(4247, 120)
(4249, 606)
(4251, 108)
(4253, 1063)
(4257, 126)
(4259, 4258)
(4261, 4260)
(4263, 84)
(4267, 400)
(4269, 474)
(4271, 2135)
(4273, 1424)
(4277, 138)
(4279, 388)
(4281, 2139)
(4283, 2141)
(4287, 1428)
(4289, 2144)
(4291, 102)
(4293, 1053)
(4297, 1432)
(4299, 4296)
(4301, 176)
(4303, 330)
(4307, 232)
(4309, 690)
(4311, 2151)
(4313, 2034)
(4317, 2157)
(4319, 264)
(4321, 1036)
(4323, 390)
(4327, 4326)
(4329, 18)
(4331, 420)
(4333, 618)
(4337, 4336)
(4339, 4338)
(4341, 1446)
(4343, 84)
(4347, 594)
(4349, 4348)
(4351, 684)
(4353, 870)
(4357, 242)
(4359, 726)
(4361, 924)
(4363, 2181)
(4367, 198)
(4369, 256)
(4371, 690)
(4373, 1093)
(4377, 162)
(4379, 2100)
(4381, 336)
(4383, 486)
(4387, 265)
(4389, 18)
(4391, 2195)
(4393, 2090)
(4397, 314)
(4399, 533)
(4401, 81)
(4403, 48)
(4407, 336)
(4409, 551)
(4411, 200)
(4413, 735)
(4417, 630)
(4419, 4410)
(4421, 4420)
(4423, 4422)
(4427, 2088)


(6449, 1612)
(6451, 2150)
(6453, 189)
(6457, 586)
(6459, 6456)
(6461, 210)
(6463, 308)
(6467, 3108)
(6469, 924)
(6471, 3231)
(6473, 6472)
(6477, 336)
(6479, 90)
(6481, 270)
(6483, 30)
(6487, 498)
(6489, 306)
(6491, 1298)
(6493, 525)
(6497, 88)
(6499, 1056)
(6501, 294)
(6503, 1392)
(6507, 270)
(6509, 3102)
(6511, 3056)
(6513, 498)
(6517, 882)
(6519, 195)
(6521, 815)
(6523, 592)
(6527, 3180)
(6529, 1088)
(6531, 930)
(6533, 46)
(6537, 2178)
(6539, 1506)
(6541, 30)
(6543, 2178)
(6547, 1091)
(6549, 174)
(6551, 3275)
(6553, 6552)
(6557, 533)
(6559, 936)
(6561, 6561)
(6563, 3281)
(6567, 198)
(6569, 1642)
(6571, 6570)
(6573, 312)
(6577, 2192)
(6579, 1008)
(6581, 1316)
(6583, 3164)
(6587, 2820)
(6589, 598)
(6591, 1014)
(6593, 3114)
(6597, 549)
(6599, 3299)
(6601, 330)
(6603, 105)
(6607, 2202)
(6609, 1101)
(6611, 300)
(6613, 1552)
(6617, 1524)
(6619, 6618)
(6621, 6618)
(6623, 534)
(6627, 6486)
(6629, 2838)
(6631, 1044)
(6633, 198)
(6637, 474)
(6639, 1659)
(6641, 1596)
(6643, 24)
(6647, 2992)
(66

(8693, 4346)
(8697, 222)
(8699, 8698)
(8701, 336)
(8703, 2898)
(8707, 4353)
(8709, 8706)
(8711, 420)
(8713, 8712)
(8717, 4158)
(8719, 4359)
(8721, 432)
(8723, 60)
(8727, 8724)
(8729, 84)
(8731, 8730)
(8733, 105)
(8737, 2912)
(8739, 8730)
(8741, 8740)
(8743, 624)
(8747, 4373)
(8749, 672)
(8751, 1458)
(8753, 8752)
(8757, 414)
(8759, 4140)
(8761, 876)
(8763, 462)
(8767, 398)
(8769, 39)
(8771, 3738)
(8773, 705)
(8777, 4290)
(8779, 22)
(8781, 8778)
(8783, 8782)
(8787, 84)
(8789, 368)
(8791, 4292)
(8793, 8784)
(8797, 1386)
(8799, 1254)
(8801, 1014)
(8803, 1467)
(8807, 8806)
(8809, 4202)
(8811, 396)
(8813, 3774)
(8817, 8814)
(8819, 8818)
(8821, 8820)
(8823, 2064)
(8827, 96)
(8829, 324)
(8831, 4415)
(8833, 88)
(8837, 4418)
(8839, 4419)
(8841, 420)
(8843, 21)
(8847, 8838)
(8849, 553)
(8851, 2158)
(8853, 678)
(8857, 208)
(8859, 984)
(8861, 8860)
(8863, 8862)
(8867, 4433)
(8869, 1260)
(8871, 4434)
(8873, 4194)
(8877, 804)
(8879, 2046)
(8881, 2173)
(8883, 1242)
(8887, 8886)
(8889, 4443)
(8891, 417

(10889, 2722)
(10891, 1210)
(10893, 1815)
(10897, 32)
(10899, 774)
(10901, 990)
(10903, 10902)
(10907, 2514)
(10909, 1212)
(10911, 909)
(10913, 4674)
(10917, 1818)
(10919, 5340)
(10921, 891)
(10923, 330)
(10927, 1554)
(10929, 1821)
(10931, 1712)
(10933, 2436)
(10937, 10936)
(10939, 10938)
(10941, 156)
(10943, 480)
(10947, 660)
(10949, 10948)
(10951, 5336)
(10953, 10944)
(10957, 2739)
(10959, 84)
(10961, 672)
(10963, 576)
(10967, 166)
(10969, 1566)
(10971, 2574)
(10973, 2743)
(10977, 10974)
(10979, 10978)
(10981, 598)
(10983, 522)
(10987, 5493)
(10989, 54)
(10991, 756)
(10993, 10992)
(10997, 4710)
(10999, 5168)
(11001, 576)
(11003, 5501)
(11007, 10998)
(11009, 108)
(11011, 66)
(11013, 1101)
(11017, 5258)
(11019, 3672)
(11021, 1802)
(11023, 600)
(11027, 5513)
(11029, 1340)
(11031, 5514)
(11033, 464)
(11037, 282)
(11039, 738)
(11041, 180)
(11043, 1836)
(11047, 11046)
(11049, 84)
(11051, 5376)
(11053, 1578)
(11057, 11056)
(11059, 11058)
(11061, 11052)
(11063, 66)
(11067, 240)
(11069, 11068

(13073, 192)
(13077, 2178)
(13079, 140)
(13081, 714)
(13083, 924)
(13087, 3124)
(13089, 2181)
(13091, 234)
(13093, 2182)
(13097, 5610)
(13099, 13098)
(13101, 198)
(13103, 13102)
(13107, 768)
(13109, 13108)
(13111, 1872)
(13113, 2070)
(13117, 252)
(13119, 3279)
(13121, 6560)
(13123, 1192)
(13127, 13126)
(13129, 2070)
(13131, 162)
(13133, 6270)
(13137, 2100)
(13139, 2814)
(13141, 3088)
(13143, 336)
(13147, 939)
(13149, 486)
(13151, 1315)
(13153, 1878)
(13157, 6438)
(13159, 2193)
(13161, 795)
(13163, 6581)
(13167, 18)
(13169, 1518)
(13171, 4390)
(13173, 6585)
(13177, 13176)
(13179, 6270)
(13181, 5628)
(13183, 4394)
(13187, 6593)
(13189, 1188)
(13191, 942)
(13193, 2158)
(13197, 1599)
(13199, 3234)
(13201, 1071)
(13203, 81)
(13207, 644)
(13209, 48)
(13211, 200)
(13213, 360)
(13217, 13216)
(13219, 4406)
(13221, 1008)
(13223, 354)
(13227, 1653)
(13229, 13228)
(13231, 260)
(13233, 600)
(13237, 60)
(13239, 2205)
(13241, 1655)
(13243, 720)
(13247, 3054)
(13249, 288)
(13251, 630)
(13253, 1064)
(1

(15357, 2559)
(15359, 7679)
(15361, 256)
(15363, 7668)
(15367, 462)
(15369, 2484)
(15371, 1818)
(15373, 1281)
(15377, 15376)
(15379, 1014)
(15381, 15372)
(15383, 15382)
(15387, 2442)
(15389, 1398)
(15391, 7695)
(15393, 366)
(15397, 1892)
(15399, 7308)
(15401, 275)
(15403, 120)
(15407, 210)
(15409, 810)
(15411, 1398)
(15413, 7706)
(15417, 5130)
(15419, 2416)
(15421, 2202)
(15423, 1248)
(15427, 7713)
(15429, 138)
(15431, 3558)
(15433, 660)
(15437, 3759)
(15439, 7719)
(15441, 7719)
(15443, 7721)
(15447, 90)
(15449, 6618)
(15451, 5150)
(15453, 144)
(15457, 420)
(15459, 15456)
(15461, 15460)
(15463, 6486)
(15467, 7733)
(15469, 2490)
(15471, 7695)
(15473, 15472)
(15477, 66)
(15479, 2464)
(15481, 112)
(15483, 198)
(15487, 7280)
(15489, 3870)
(15491, 3318)
(15493, 7746)
(15497, 1192)
(15499, 32)
(15501, 5166)
(15503, 1254)
(15507, 2583)
(15509, 3576)
(15511, 7755)
(15513, 330)
(15517, 7598)
(15519, 246)
(15521, 656)
(15523, 2394)
(15527, 15526)
(15529, 1898)
(15531, 2490)
(15533, 3318)
(15537,

(17639, 4260)
(17641, 1914)
(17643, 2940)
(17647, 630)
(17649, 117)
(17651, 4176)
(17653, 966)
(17657, 17656)
(17659, 5886)
(17661, 2436)
(17663, 8304)
(17667, 450)
(17669, 17668)
(17671, 215)
(17673, 168)
(17677, 1606)
(17679, 4305)
(17681, 1105)
(17683, 8841)
(17687, 2112)
(17689, 2394)
(17691, 17688)
(17693, 2040)
(17697, 8304)
(17699, 402)
(17701, 570)
(17703, 252)
(17707, 8853)
(17709, 17706)
(17711, 396)
(17713, 17712)
(17717, 138)
(17719, 1932)
(17721, 1602)
(17723, 717)
(17727, 2790)
(17729, 1108)
(17731, 1776)
(17733, 8448)
(17737, 17736)
(17739, 1944)
(17741, 4368)
(17743, 806)
(17747, 8873)
(17749, 17748)
(17751, 480)
(17753, 2160)
(17757, 8874)
(17759, 1218)
(17761, 8880)
(17763, 285)
(17767, 324)
(17769, 2961)
(17771, 4098)
(17773, 2538)
(17777, 1428)
(17779, 4246)
(17781, 17778)
(17783, 17782)
(17787, 462)
(17789, 17788)
(17791, 8895)
(17793, 17766)
(17797, 222)
(17799, 1392)
(17801, 7626)
(17803, 936)
(17807, 17806)
(17809, 1618)
(17811, 17802)
(17813, 8694)
(17817, 1781

(19929, 24)
(19931, 4716)
(19933, 1605)
(19937, 2848)
(19939, 546)
(19941, 8976)
(19943, 42)
(19947, 540)
(19949, 19948)
(19951, 140)
(19953, 2214)
(19957, 2850)
(19959, 9978)
(19961, 9980)
(19963, 3327)
(19967, 2430)
(19969, 3150)
(19971, 1422)
(19973, 9986)
(19977, 19974)
(19979, 19978)
(19981, 1092)
(19983, 6660)
(19987, 286)
(19989, 6660)
(19991, 9995)
(19993, 19992)
(19997, 9998)
(19999, 408)
(20001, 9744)
(20003, 1230)
(20007, 162)
(20009, 848)
(20011, 6670)
(20013, 2856)
(20017, 540)
(20019, 6672)
(20021, 1540)
(20023, 6674)
(20027, 8580)
(20029, 6676)
(20031, 606)
(20033, 66)
(20037, 3339)
(20039, 3220)
(20041, 1428)
(20043, 9360)
(20047, 20046)


In [196]:
A_kds = Dict([(k, sort(map(first,ds))) for (k,ds) in groupby(last,A)])
A_kds_list = [(k,ds) for (k,ds) in sort(pairs(A_kds))]
@print_seq A_kds_list
length(A_kds_list)

(2, [11])
(3, [3, 37, 111])
(4, [101, 1111])
(5, [41, 271, 11111])
(6, [7, 13, 21, 33, 39, 77, 91, 143, 231, 259, 273, 407, 429, 481, 777, 1001, 1221, 1443, 2849, 3003, 3367, 5291, 8547, 10101, 15873])
(7, [239, 4649])
(8, [73, 137, 803, 1507, 7373, 10001, 13837])
(9, [9, 333])
(10, [451, 2981, 9091])
(12, [303, 707, 1313, 2121, 3333, 3737, 3939, 7777, 9191, 9901, 11211, 14443])
(13, [53, 79, 4187])
(14, [2629])
(15, [31, 93, 123, 813, 1147, 1271, 1517, 3441, 3813, 4551, 8401, 10027])
(16, [17, 187, 1241, 1717, 2329, 13651, 18887])
(18, [19, 57, 63, 99, 117, 133, 171, 209, 247, 399, 627, 693, 703, 741, 819, 1197, 1287, 1463, 1729, 1881, 2109, 2223, 2331, 2717, 3663, 4329, 4389, 4921, 5187, 6327, 7733, 8151, 9009, 9139, 13167, 14763, 15561, 19019])
(20, [3541, 4141])
(21, [43, 129, 717, 1591, 1933, 4773, 5799, 8843, 10277, 13947])
(22, [23, 121, 253, 2783, 4093, 8779])
(24, [219, 411, 511, 949, 959, 1533, 1781, 2409, 2701, 2847, 2877, 4521, 5069, 5343, 5621, 6643, 8103, 10439, 10549, 12

(232, [233, 2563, 4307, 8083, 13747, 17009])
(233, [467])
(234, [1007, 1413, 1501, 1521, 2983, 3021, 3211, 3339, 4503, 4977, 5247, 6201, 7049, 7731, 7821, 8949, 9063, 9243, 9633, 9891, 10507, 10647, 11077, 13091, 13509, 15543, 16321, 16511, 16731, 18369, 19513])
(237, [951, 11729, 18489])
(239, [479])
(240, [527, 1037, 1581, 2091, 3111, 3587, 3689, 4097, 4879, 5797, 6851, 7259, 9061, 10761, 11067, 11407, 12291, 13481, 13821, 14637, 17391, 19499])
(242, [1331, 4357])
(243, [243, 8991])
(246, [581, 739, 1079, 1743, 2217, 2739, 3237, 5173, 6391, 7553, 8129, 8617, 9607, 11869, 15519, 16003, 19173])
(247, [1483])
(248, [1489, 16379])
(249, [4483, 13449])
(250, [8261])
(252, [261, 551, 1009, 1653, 1827, 2529, 2871, 3027, 3393, 3857, 4959, 5339, 6061, 7063, 7163, 9081, 9657, 11099, 11223, 11571, 12789, 13117, 16017, 17703, 18183, 19171])
(253, [1013, 3037])
(254, [3557])
(255, [19003])
(256, [257, 2827, 4369, 15361, 18761])
(258, [1211, 2249, 3633, 5709, 6747, 13321, 15743])
(259, [2591])
(26

(882, [1773, 2647, 3087, 3743, 6181, 6517, 7941, 9713, 11229, 11479, 12411, 16777, 18529, 18543, 19503, 19551])
(886, [887, 2659, 9757])
(888, [16279])
(890, [7339])
(891, [5427, 10921, 16119])
(893, [1787])
(894, [6261, 14609, 18781])
(895, [14719])
(897, [1797, 14681])
(900, [1801, 5403, 5409, 11419, 12607, 16209, 19811])
(902, [1909, 10043])
(903, [1849, 5547, 7439])
(904, [16571])
(906, [6349, 11791, 19047])
(907, [5443])
(909, [3637, 10911])
(910, [9301, 10021])
(912, [3893, 11679])
(915, [5493, 7323])
(916, [2749])
(918, [2781, 6433, 10109, 11947, 17461, 19299, 19467])
(924, [1943, 2001, 3827, 4361, 4621, 4669, 5829, 6469, 7743, 8671, 10527, 11303, 11481, 13083, 13601, 13863, 14007, 18067, 18827, 19389, 19407])
(926, [5557])
(927, [9279])
(928, [9281])
(930, [2177, 4043, 6531, 6727, 10263, 10571, 11563, 12129, 12493, 15293])
(931, [16759])
(933, [1867, 3733, 5601, 11199])
(935, [1871])
(936, [937, 2811, 2817, 5947, 6559, 8433, 10307, 12181, 17803, 17841, 19677, 19719])
(938, [187

(1984, [5953])
(1986, [13909])
(1988, [9941, 16501])
(1989, [3987, 16271])
(1992, [5979, 13951])
(1995, [8213])
(1998, [6021, 13993])
(2000, [12767, 16001])
(2002, [8009, 12581])
(2007, [12043])
(2010, [6033, 14077])
(2013, [4027, 12081])
(2015, [16483])
(2016, [2017, 6051, 6057, 12787, 14119, 18153])
(2020, [8081])
(2024, [4049])
(2025, [12231])
(2026, [12157])
(2028, [2029, 6087, 14203])
(2029, [8117])
(2030, [4189, 6091])
(2032, [8653])
(2034, [4313, 12939, 14301])
(2039, [4079])
(2040, [4083, 9527, 17693])
(2044, [8497])
(2046, [4781, 8187, 8579, 8879, 14343, 19103])
(2052, [6183])
(2055, [4111, 12333])
(2058, [2401, 4119, 7203, 9611, 17849])
(2060, [16481])
(2062, [2063])
(2064, [4129, 8823, 12387, 17561])
(2066, [10331])
(2067, [8427, 17013])
(2068, [2069])
(2070, [6219, 13113, 13129, 17343])
(2073, [8293])
(2076, [6229, 18687])
(2078, [4157])
(2082, [14581])
(2085, [12511])
(2088, [2097, 4427, 13281, 14679])
(2090, [4393, 17179])
(2094, [10473])
(2096, [4471, 17833])
(2097, [420

(3666, [3669, 8561, 15899])
(3668, [7627])
(3672, [3673, 11019])
(3674, [18371])
(3681, [14733])
(3684, [3687, 8603, 15977])
(3696, [7571, 7751, 7797, 11091, 16813, 18193, 19159])
(3699, [14797])
(3700, [3701])
(3704, [11113])
(3708, [3709, 11127, 18541])
(3714, [17339])
(3717, [7443])
(3718, [15571])
(3726, [3727, 3807, 7661, 11181, 11259])
(3728, [7939])
(3729, [15209])
(3738, [7477, 7697, 8771, 11217])
(3740, [18701])
(3746, [18731])
(3753, [7507, 15013])
(3758, [7517])
(3759, [15437])
(3760, [15997])
(3761, [7523])
(3762, [3771, 7961, 8303])
(3763, [15053])
(3766, [3767])
(3770, [7729])
(3771, [7551])
(3773, [7547])
(3774, [3777, 8813, 16367])
(3778, [3779])
(3779, [7559])
(3780, [7739, 11367, 15689])
(3790, [15161])
(3792, [11379, 16167])
(3795, [7591])
(3806, [7981])
(3810, [15241, 15243])
(3818, [7849])
(3819, [7639, 15277])
(3820, [3821])
(3821, [7643])
(3822, [11469, 18179])
(3824, [8143])
(3828, [15753])
(3832, [3833])
(3840, [7967, 15677])
(3842, [11527, 19211])
(3846, [3847

(6350, [19051])
(6352, [6353])
(6356, [19069])
(6362, [19087])
(6366, [6367, 19101])
(6372, [19143])
(6378, [19137])
(6388, [6389])
(6390, [19179])
(6394, [19183, 19321])
(6395, [12791])
(6404, [12809])
(6405, [12813])
(6408, [19233])
(6417, [12843])
(6420, [6423, 12841, 14987, 19263])
(6423, [12849])
(6432, [12867])
(6438, [13157])
(6446, [13501])
(6450, [19353])
(6453, [12907, 12933])
(6455, [12911])
(6456, [6459, 15071])
(6458, [12917])
(6461, [12923])
(6462, [13661])
(6471, [12951])
(6472, [6473])
(6474, [19423])
(6479, [12959])
(6480, [13787])
(6486, [6627, 13301, 15463])
(6498, [6859])
(6501, [13003])
(6510, [15239, 19531])
(6512, [13639])
(6518, [13037])
(6521, [13043])
(6524, [13543])
(6534, [19629])
(6552, [6553, 19659])
(6554, [13393])
(6560, [13121])
(6561, [6561])
(6570, [6571, 19713])
(6576, [13991, 19731])
(6578, [13777])
(6581, [13163])
(6585, [13173])
(6592, [19777])
(6593, [13187])
(6608, [14059])
(6612, [13511])
(6618, [6619, 6621, 15449, 19857])
(6620, [19861])
(6630

(10650, [10651])
(10656, [10657])
(10662, [10663])
(10686, [10687])
(10690, [10691])
(10708, [10709])
(10728, [10737])
(10738, [10739])
(10740, [10743])
(10776, [10779])
(10780, [10781])
(10788, [10789])
(10846, [10847])
(10848, [10851])
(10858, [10859])
(10860, [10861])
(10866, [10869])
(10902, [10903])
(10936, [10937])
(10938, [10939])
(10944, [10953])
(10948, [10949])
(10974, [10977])
(10978, [10979])
(10992, [10993])
(10998, [11007])
(11046, [11047])
(11052, [11061])
(11056, [11057])
(11058, [11059])
(11068, [11069])
(11100, [11103])
(11130, [11131])
(11148, [11149])
(11170, [11171])
(11176, [11177])
(11178, [11421])
(11256, [11257])
(11272, [11273])
(11286, [11287, 11313])
(11298, [11299, 11301])
(11322, [11331])
(11334, [11337])
(11352, [11353])
(11382, [11383])
(11392, [11393])
(11422, [11423])
(11446, [11447])
(11460, [11463])
(11496, [11497, 11499])
(11502, [11503])
(11548, [11549])
(11578, [11579])
(11586, [11589])
(11592, [11593])
(11616, [11617])
(11620, [11621])
(11632, [1

3485

In [197]:
for p in take(10^3, Lazy.primes)
    println("$p=> $(first_k(Rs_lazy,p))")
end     

2=> nothing
3=> 3
5=> nothing
7=> 6
11=> 2
13=> 6
17=> 16
19=> 18
23=> 22
29=> 28
31=> 15
37=> 3
41=> 5
43=> 21
47=> 46
53=> 13
59=> 58
61=> 60
67=> 33
71=> 35
73=> 8
79=> 13
83=> 41
89=> 44
97=> 96
101=> 4
103=> 34
107=> 53
109=> 108
113=> 112
127=> 42
131=> 130
137=> 8
139=> 46
149=> 148
151=> 75
157=> 78
163=> 81
167=> 166
173=> 43
179=> 178
181=> 180
191=> 95
193=> 192
197=> 98
199=> 99
211=> 30
223=> 222
227=> 113
229=> 228
233=> 232
239=> 7
241=> 30
251=> 50
257=> 256
263=> 262
269=> 268
271=> 5
277=> 69
281=> 28
283=> 141
293=> 146
307=> 153
311=> 155
313=> 312
317=> 79
331=> 110
337=> 336
347=> 173
349=> 116
353=> 32
359=> 179
367=> 366
373=> 186
379=> 378
383=> 382
389=> 388
397=> 99
401=> 200
409=> 204
419=> 418
421=> 140
431=> 215
433=> 432
439=> 219
443=> 221
449=> 32
457=> 152
461=> 460
463=> 154
467=> 233
479=> 239
487=> 486
491=> 490
499=> 498
503=> 502
509=> 508
521=> 52
523=> 261
541=> 540
547=> 91
557=> 278
563=> 281
569=> 284
571=> 570
577=> 576
587=> 293
593=> 592
5

5741=> 5740
5743=> 5742
5749=> 5748
5779=> 5778
5783=> 5782
5791=> 965
5801=> 1450
5807=> 5806
5813=> 2906
5821=> 5820
5827=> 2913
5839=> 2919
5843=> 2921
5849=> 1462
5851=> 1950
5857=> 5856
5861=> 5860
5867=> 2933
5869=> 5868
5879=> 2939
5881=> 2940
5897=> 5896
5903=> 5902
5923=> 2961
5927=> 5926
5939=> 5938
5953=> 1984
5981=> 5980
5987=> 2993
6007=> 858
6011=> 6010
6029=> 6028
6037=> 3018
6043=> 3021
6047=> 6046
6053=> 3026
6067=> 3033
6073=> 6072
6079=> 1013
6089=> 761
6091=> 2030
6101=> 1220
6113=> 6112
6121=> 3060
6131=> 6130
6133=> 1533
6143=> 6142
6151=> 1025
6163=> 79
6173=> 3086
6197=> 3098
6199=> 3099
6203=> 443
6211=> 6210
6217=> 6216
6221=> 6220
6229=> 2076
6247=> 6246
6257=> 6256
6263=> 6262
6269=> 6268
6271=> 1045
6277=> 1569
6287=> 6286
6299=> 94
6301=> 6300
6311=> 3155
6317=> 3158
6323=> 3161
6329=> 3164
6337=> 6336
6343=> 6342
6353=> 6352
6359=> 3179
6361=> 1590
6367=> 6366
6373=> 1062
6379=> 2126
6389=> 6388
6397=> 78
6421=> 2140
6427=> 1071
6449=> 1612
6451=> 2150
64

In [198]:
for p in take(10^2, ds)
    #println("$p, $(A[p]), $(isprime(p)), $(R(p-1) % p)")
    println("$p=> $(first_k(Rs_lazy,p)), $(R(p-1) % p)")
end     

3=> 3, 2
7=> 6, 0
9=> 9, 8
11=> 2, 0
13=> 6, 0
17=> 16, 0
19=> 18, 0
21=> 6, 11
23=> 22, 0
27=> 27, 8
29=> 28, 0
31=> 15, 0
33=> 6, 11
37=> 3, 0
39=> 6, 11
41=> 5, 0
43=> 21, 0
47=> 46, 0
49=> 42, 28
51=> 48, 11
53=> 13, 0
57=> 18, 11
59=> 58, 0
61=> 60, 0
63=> 18, 53
67=> 33, 0
69=> 66, 11
71=> 35, 0
73=> 8, 0
77=> 6, 33
79=> 13, 0
81=> 81, 8
83=> 41, 0
87=> 84, 11
89=> 44, 0
91=> 6, 0
93=> 15, 11
97=> 96, 0
99=> 18, 44
101=> 4, 0
103=> 34, 0
107=> 53, 0
109=> 108, 0
111=> 3, 11
113=> 112, 0
117=> 18, 89
119=> 48, 33
121=> 22, 55
123=> 15, 11
127=> 42, 0
129=> 21, 11
131=> 130, 0
133=> 18, 56
137=> 8, 0
139=> 46, 0
141=> 138, 11
143=> 6, 110
147=> 42, 32
149=> 148, 0
151=> 75, 0
153=> 144, 98
157=> 78, 0
159=> 39, 11
161=> 66, 159
163=> 81, 0
167=> 166, 0
169=> 78, 156
171=> 18, 44
173=> 43, 0
177=> 174, 11
179=> 178, 0
181=> 180, 0
183=> 60, 11
187=> 16, 121
189=> 54, 116
191=> 95, 0
193=> 192, 0
197=> 98, 0
199=> 99, 0
201=> 33, 11
203=> 84, 12
207=> 198, 179
209=> 18, 22
211=> 30, 

Looks like for R(d-1) % d ==0 | d <-  prime ds 

`pds are the prime 'ds' also the odd primes`

In [46]:
pds = tail(Lazy.primes);

`Return the gcd of the 'ks' associated the multiples of 'd' found in 'A'`

In [199]:
ks_gcd(A,d) = @>> collect(pairs(A)) filter(p-> p[1]%d==0 ) map(p-> p[2]) gcd 

ks_gcd (generic function with 1 method)

In [200]:
D = 3 # (3)
for i in take(20, ds)
    println("($D*$i), ($(i*D), $(A[i*D])), ($(factors(A[i*D])))")
end

(3*3), (9, 9), (3^2)
(3*7), (21, 6), (2 ⋅ 3)
(3*9), (27, 27), (3^3)
(3*11), (33, 6), (2 ⋅ 3)
(3*13), (39, 6), (2 ⋅ 3)
(3*17), (51, 48), (2^4 ⋅ 3)
(3*19), (57, 18), (2 ⋅ 3^2)
(3*21), (63, 18), (2 ⋅ 3^2)
(3*23), (69, 66), (2 ⋅ 3 ⋅ 11)
(3*27), (81, 81), (3^4)
(3*29), (87, 84), (2^2 ⋅ 3 ⋅ 7)
(3*31), (93, 15), (3 ⋅ 5)
(3*33), (99, 18), (2 ⋅ 3^2)
(3*37), (111, 3), (3)
(3*39), (117, 18), (2 ⋅ 3^2)
(3*41), (123, 15), (3 ⋅ 5)
(3*43), (129, 21), (3 ⋅ 7)
(3*47), (141, 138), (2 ⋅ 3 ⋅ 23)
(3*49), (147, 42), (2 ⋅ 3 ⋅ 7)
(3*51), (153, 144), (2^4 ⋅ 3^2)


In [201]:
println(ks_gcd(A,3))

3


In [202]:
D = 7 # (2,3)
for i in take(20, ds)
    println("($D*$i), ($(i*D), $(A[i*D])), ($(factors(A[i*D])))")
end

(7*3), (21, 6), (2 ⋅ 3)
(7*7), (49, 42), (2 ⋅ 3 ⋅ 7)
(7*9), (63, 18), (2 ⋅ 3^2)
(7*11), (77, 6), (2 ⋅ 3)
(7*13), (91, 6), (2 ⋅ 3)
(7*17), (119, 48), (2^4 ⋅ 3)
(7*19), (133, 18), (2 ⋅ 3^2)
(7*21), (147, 42), (2 ⋅ 3 ⋅ 7)
(7*23), (161, 66), (2 ⋅ 3 ⋅ 11)
(7*27), (189, 54), (2 ⋅ 3^3)
(7*29), (203, 84), (2^2 ⋅ 3 ⋅ 7)
(7*31), (217, 30), (2 ⋅ 3 ⋅ 5)
(7*33), (231, 6), (2 ⋅ 3)
(7*37), (259, 6), (2 ⋅ 3)
(7*39), (273, 6), (2 ⋅ 3)
(7*41), (287, 30), (2 ⋅ 3 ⋅ 5)
(7*43), (301, 42), (2 ⋅ 3 ⋅ 7)
(7*47), (329, 138), (2 ⋅ 3 ⋅ 23)
(7*49), (343, 294), (2 ⋅ 3 ⋅ 7^2)
(7*51), (357, 48), (2^4 ⋅ 3)


In [203]:
println(ks_gcd(A,7))

6


In [204]:
for d in take(10^2,filter(isprime,ds))
    println("$d => $(ks_gcd(A,d)), $(A[d])")
end

3 => 3, 3
7 => 6, 6
11 => 2, 2
13 => 6, 6
17 => 16, 16
19 => 18, 18
23 => 22, 22
29 => 28, 28
31 => 15, 15
37 => 3, 3
41 => 5, 5
43 => 21, 21
47 => 46, 46
53 => 13, 13
59 => 58, 58
61 => 60, 60
67 => 33, 33
71 => 35, 35
73 => 8, 8
79 => 13, 13
83 => 41, 41
89 => 44, 44
97 => 96, 96
101 => 4, 4
103 => 34, 34
107 => 53, 53
109 => 108, 108
113 => 112, 112
127 => 42, 42
131 => 130, 130
137 => 8, 8
139 => 46, 46
149 => 148, 148
151 => 75, 75
157 => 78, 78
163 => 81, 81
167 => 166, 166
173 => 43, 43
179 => 178, 178
181 => 180, 180
191 => 95, 95
193 => 192, 192
197 => 98, 98
199 => 99, 99
211 => 30, 30
223 => 222, 222
227 => 113, 113
229 => 228, 228
233 => 232, 232
239 => 7, 7
241 => 30, 30
251 => 50, 50
257 => 256, 256
263 => 262, 262
269 => 268, 268
271 => 5, 5
277 => 69, 69
281 => 28, 28
283 => 141, 141
293 => 146, 146
307 => 153, 153
311 => 155, 155
313 => 312, 312
317 => 79, 79
331 => 110, 110
337 => 336, 336
347 => 173, 173
349 => 116, 116
353 => 32, 32
359 => 179, 179
367 => 366, 366
3

#### ks_gcd(A,d) = A[d] when d is prime and d <- ds

In other words, for all `p` that are odd primes, there is an `A[p] = k`, such that `A[p*d] = m*k`, where `p*d <- ds`. For example, for `p = 11`, `A[11] = 2`, `A[33]= 6`, `A[77] = 6`, `A[99] = 18`, `A[121] = 22`. Each of the ks is divisible by 2.

In [205]:
for d in take(10^3,filter(!isprime,ds))
    println("$d => $(prod(ks_gcd(A,d))), $(A[d])")
end

9 => 9, 9
21 => 6, 6
27 => 27, 27
33 => 6, 6
39 => 6, 6
49 => 42, 42
51 => 48, 48
57 => 18, 18
63 => 18, 18
69 => 66, 66
77 => 6, 6
81 => 81, 81
87 => 84, 84
91 => 6, 6
93 => 15, 15
99 => 18, 18
111 => 3, 3
117 => 18, 18
119 => 48, 48
121 => 22, 22
123 => 15, 15
129 => 21, 21
133 => 18, 18
141 => 138, 138
143 => 6, 6
147 => 42, 42
153 => 144, 144
159 => 39, 39
161 => 66, 66
169 => 78, 78
171 => 18, 18
177 => 174, 174
183 => 60, 60
187 => 16, 16
189 => 54, 54
201 => 33, 33
203 => 84, 84
207 => 198, 198
209 => 18, 18
213 => 105, 105
217 => 30, 30
219 => 24, 24
221 => 48, 48
231 => 6, 6
237 => 39, 39
243 => 243, 243
247 => 18, 18
249 => 123, 123
253 => 22, 22
259 => 6, 6
261 => 252, 252
267 => 132, 132
273 => 6, 6
279 => 45, 45
287 => 30, 30
289 => 272, 272
291 => 96, 96
297 => 54, 54
299 => 66, 66
301 => 42, 42
303 => 12, 12
309 => 102, 102
319 => 28, 28
321 => 159, 159
323 => 144, 144
327 => 108, 108
329 => 138, 138
333 => 9, 9
339 => 336, 336
341 => 30, 30
343 => 294, 294
351 => 54, 54

2151 => 63, 63
2157 => 1077, 1077
2159 => 336, 336
2163 => 102, 102
2167 => 98, 98
2169 => 90, 90
2171 => 498, 498
2173 => 65, 65
2177 => 930, 930
2181 => 726, 726
2183 => 174, 174
2187 => 2187, 2187
2189 => 198, 198
2191 => 312, 312
2193 => 336, 336
2197 => 1014, 1014
2199 => 183, 183
2201 => 105, 105
2209 => 2162, 2162
2211 => 66, 66
2217 => 246, 246
2219 => 474, 474
2223 => 18, 18
2227 => 1040, 1040
2229 => 2226, 2226
2231 => 1056, 1056
2233 => 84, 84
2241 => 1107, 1107
2247 => 318, 318
2249 => 258, 258
2253 => 375, 375
2257 => 60, 60
2259 => 450, 450
2261 => 144, 144
2263 => 120, 120
2271 => 27, 27
2277 => 198, 198
2279 => 273, 273
2283 => 1140, 1140
2289 => 108, 108
2291 => 364, 364
2299 => 198, 198
2301 => 174, 174
2303 => 966, 966
2307 => 192, 192
2313 => 2304, 2304
2317 => 330, 330
2319 => 579, 579
2321 => 30, 30
2323 => 44, 44
2327 => 534, 534
2329 => 16, 16
2331 => 18, 18
2337 => 90, 90
2343 => 210, 210
2349 => 2268, 2268
2353 => 180, 180
2359 => 336, 336
2361 => 393, 393
236

    For A[d], where d is a composite, lets try to relate `A[d] to A[f_1] * A[f_2] ... f_i <- factors(d)`

In [552]:
cs = filter(!isprime,ds);
take(10,cs)

(9 21 27 33 39 49 51 57 63 69)

In [567]:
for c in take(10,cs)
    println("$c, $(A[c]), $(factors(c)), $(map(f->(f, A[f]), collect(keys(factor(c)))))")
end

9, 9, 3^2, Tuple{Int64,Int64}[(3, 3)]
21, 6, 3 ⋅ 7, Tuple{Int64,Int64}[(3, 3), (7, 6)]
27, 27, 3^3, Tuple{Int64,Int64}[(3, 3)]
33, 6, 3 ⋅ 11, Tuple{Int64,Int64}[(3, 3), (11, 2)]
39, 6, 3 ⋅ 13, Tuple{Int64,Int64}[(3, 3), (13, 6)]
49, 42, 7^2, Tuple{Int64,Int64}[(7, 6)]
51, 48, 3 ⋅ 17, Tuple{Int64,Int64}[(3, 3), (17, 16)]
57, 18, 3 ⋅ 19, Tuple{Int64,Int64}[(3, 3), (19, 18)]
63, 18, 3^2 ⋅ 7, Tuple{Int64,Int64}[(3, 3), (7, 6)]
69, 66, 3 ⋅ 23, Tuple{Int64,Int64}[(3, 3), (23, 22)]


In [560]:
factor(60)

2^2 ⋅ 3 ⋅ 5

In [564]:
collect(keys(factor(60)))

3-element Array{Int64,1}:
 2
 3
 5

Why are there so many examples of A[d] - k = 1?

In [262]:
for d in take(100,filter(!isprime,ds))
    println("$d, $(R(d-1) % d)")
end

9, 8
21, 11
27, 8
33, 11
39, 11
49, 28
51, 11
57, 11
63, 53
69, 11
77, 33
81, 8
87, 11
91, 0
93, 11
99, 44
111, 11
117, 89
119, 33
121, 55
123, 11
129, 11
133, 56
141, 11
143, 110
147, 32
153, 98
159, 11
161, 159
169, 156
171, 44
177, 11
183, 11
187, 121
189, 116
201, 11
203, 12
207, 179
209, 22
213, 11
217, 7
219, 11
221, 58
231, 11
237, 11
243, 170
247, 26
249, 11
253, 132
259, 0
261, 80
267, 11
273, 11
279, 215
287, 124
289, 272
291, 11
297, 143
299, 32
301, 42
303, 11
309, 11
319, 297
321, 11
323, 317
327, 11
329, 285
333, 233
339, 11
341, 121
343, 175
351, 89
357, 74
361, 152
363, 176
369, 152
371, 341
377, 305
381, 11
387, 341
391, 169
393, 11
399, 11
403, 52
407, 297
411, 11
413, 250
417, 11
423, 170
427, 91
429, 11
437, 351
441, 179
447, 11
451, 0
453, 11
459, 359
469, 427
471, 11
473, 55


In [273]:
for d in take(100,filter(isprime,ds))
    println("$d, $(R(d-1) % d)")
end

3, 2
7, 0
11, 0
13, 0
17, 0
19, 0
23, 0
29, 0
31, 0
37, 0
41, 0
43, 0
47, 0
53, 0
59, 0
61, 0
67, 0
71, 0
73, 0
79, 0
83, 0
89, 0
97, 0
101, 0
103, 0
107, 0
109, 0
113, 0
127, 0
131, 0
137, 0
139, 0
149, 0
151, 0
157, 0
163, 0
167, 0
173, 0
179, 0
181, 0
191, 0
193, 0
197, 0
199, 0
211, 0
223, 0
227, 0
229, 0
233, 0
239, 0
241, 0
251, 0
257, 0
263, 0
269, 0
271, 0
277, 0
281, 0
283, 0
293, 0
307, 0
311, 0
313, 0
317, 0
331, 0
337, 0
347, 0
349, 0
353, 0
359, 0
367, 0
373, 0
379, 0
383, 0
389, 0
397, 0
401, 0
409, 0
419, 0
421, 0
431, 0
433, 0
439, 0
443, 0
449, 0
457, 0
461, 0
463, 0
467, 0
479, 0
487, 0
491, 0
499, 0
503, 0
509, 0
521, 0
523, 0
541, 0
547, 0
557, 0


Interestingly, in A, there are many A[d] = k, where d - A[d] = 1. Looking at these results, it appears that if d <- primes, that R[p-1] % P == 0 all p. For example, R(10) % 11 = 0.  Note the converse in not necessarily true, i.e., for composites, mod(A[d], p-1) is generally not equal to 0, but composite numbers do exist satisfying this condition. 

#### R[p - 1]  % p == 0 for all p > 3. 

In [279]:
for d in take(1000,filter(isprime,ds))
    println("$d, $(A[d]) $(d-A[d]==1 ? "*" : "")")
end

3, 3 
7, 6 *
11, 2 
13, 6 
17, 16 *
19, 18 *
23, 22 *
29, 28 *
31, 15 
37, 3 
41, 5 
43, 21 
47, 46 *
53, 13 
59, 58 *
61, 60 *
67, 33 
71, 35 
73, 8 
79, 13 
83, 41 
89, 44 
97, 96 *
101, 4 
103, 34 
107, 53 
109, 108 *
113, 112 *
127, 42 
131, 130 *
137, 8 
139, 46 
149, 148 *
151, 75 
157, 78 
163, 81 
167, 166 *
173, 43 
179, 178 *
181, 180 *
191, 95 
193, 192 *
197, 98 
199, 99 
211, 30 
223, 222 *
227, 113 
229, 228 *
233, 232 *
239, 7 
241, 30 
251, 50 
257, 256 *
263, 262 *
269, 268 *
271, 5 
277, 69 
281, 28 
283, 141 
293, 146 
307, 153 
311, 155 
313, 312 *
317, 79 
331, 110 
337, 336 *
347, 173 
349, 116 
353, 32 
359, 179 
367, 366 *
373, 186 
379, 378 *
383, 382 *
389, 388 *
397, 99 
401, 200 
409, 204 
419, 418 *
421, 140 
431, 215 
433, 432 *
439, 219 
443, 221 
449, 32 
457, 152 
461, 460 *
463, 154 
467, 233 
479, 239 
487, 486 *
491, 490 *
499, 498 *
503, 502 *
509, 508 *
521, 52 
523, 261 
541, 540 *
547, 91 
557, 278 
563, 281 
569, 284 
571, 570 *
577, 576 *
587, 

In [292]:
for g in 1000:1000:20000
    g,d,k=k_over(A,g)
    println("$g, $d, $k, $(isprime(d)), $(d-k==1 ? "*" : "")")
end

1000, 1017, 1008, false, 
2000, 2017, 2016, true, *
3000, 3011, 3010, true, *
4000, 4007, 4006, true, *
5000, 5021, 5020, true, *
6000, 6011, 6010, true, *
7000, 7017, 7014, false, 
8000, 8017, 8016, true, *
9000, 9011, 9010, true, *
10000, 10007, 10006, true, *
11000, 11047, 11046, true, *
12000, 12011, 12010, true, *
13000, 13007, 13006, true, *
14000, 14011, 14010, true, *
15000, 15017, 15016, true, *
16000, 16007, 16006, true, *
17000, 17007, 17004, false, 
18000, 18033, 18030, false, 
19000, 19059, 19056, false, 
20000, 20047, 20046, true, *


#### d >= A[d] for all d. There are d for which A[d] = d

In [17]:
function first_k(Rs,d) 
    for (k,R) in enumerate(Rs)
        if R%d == 0
            return(k)
        end
    end
    return("nothing")
end

first_k (generic function with 1 method)

In [308]:
function k_over(A,goal) 
    for d in sort(collect(keys(A)))
        if A[d] > goal
            return((goal, d, A[d]))
            break
        end
    end
    println("nothing")
end

k_over (generic function with 1 method)

In [549]:
# Original
function k_over(goal)
    d_,r_= (0,0)
    for d in dropwhile(d-> d<=goal, ds)
        println("($d_, $r_) $(isprime(d) ? "*" : "")")
        r_ > goal && return(d_,r_)
        for r in 2:d
            #println(r)
            R_ = R(r)
            if R_%d == 0
                d_,r_ = d,r
                break
            end
        end
    end
    return("nothing")
end

k_over (generic function with 2 methods)

In [547]:
# Filtering for primes
function k_over(goal)
    d_,r_= (0,0)
    for d in filter(!isprime,dropwhile(d-> d<=goal, ds))
        println("($d_, $r_) $(isprime(d) ? "*" : "")")
        r_ > goal && return(d_,r_)
        for r in 2:d
            #println(r)
            R_ = R(r)
            if R_%d == 0
                d_,r_ = d,r
                break
            end
        end
    end
    return("nothing")
end

k_over (generic function with 2 methods)

In [545]:
k_over(10^6)

(0, 0) 
(1000001, 12) 
(1000007, 68964) 
(1000009, 124538) 
(1000011, 333336) 
(1000013, 35526) 
(1000017, 159390) 
(1000019, 27186) 
(1000021, 90910) 


InterruptException: InterruptException:

In [456]:
for g in [10^i for i in 1:5]
    d,k = k_over(g)
    println("=>$g, ($d, $k) $(isprime(d) ? "*" : "") ")
end

(0, 0)
(11, 2)
(13, 6)
(17, 16)
=>10, (17, 16) * 
(0, 0)
(101, 4)
(103, 34)
(107, 53)
(109, 108)
=>100, (109, 108) * 
(0, 0)
(1001, 6)
(1003, 464)
(1007, 234)
(1009, 252)
(1011, 336)
(1013, 253)
(1017, 1008)
=>1000, (1017, 1008)  
(0, 0)
(10001, 8)
(10003, 1428)
(10007, 10006)
=>10000, (10007, 10006) * 
(0, 0)
(100001, 10)
(100003, 50001)
(100007, 9888)
(100009, 546)
(100011, 624)
(100013, 16490)
(100017, 33336)
(100019, 100018)
=>100000, (100019, 100018) * 


In [405]:
mps = primes(10^6,10^6 + 100)

6-element Array{Int64,1}:
 1000003
 1000033
 1000037
 1000039
 1000081
 1000099

In [464]:
k_over(10^6)

(0, 0)
(1000001, 12)


InterruptException: InterruptException: